# Quick Start: Azure AI Foundry SDK

### Step 1: Create a new agent
This step shows how to create an Azure AI Agent using the Foundry SDK, attach tools, and start a conversation thread. The code demonstrates agent creation, thread creation, sending a message, running the agent, and retrieving responses.

* [Azure AI Agents client library for Python - version 1.0.2](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python)
* [Azure AI Projects client library for Python - version 1.0.0b12](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview)

In [5]:
# --- Standard Library Imports ---
import os
import sys
from dotenv import load_dotenv

# --- Azure AI Foundry SDK for AI agents and services ---
from azure.ai.agents.models import CodeInterpreterTool, FilePurpose, FileSearchTool, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load the environment variables from a .env file
if not load_dotenv('../.env', override=True):
    load_dotenv(override=True)

print(f"Project Endpoint: {os.getenv('AZURE_AI_AGENT_ENDPOINT')}")
print(f"Model Deployment Name: {os.getenv('AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME')}")

# Check the current working Python execution environment
print(f"Current Python executable: {sys.executable}")

Project Endpoint: https://ai-foundry-hackathon-team55.services.ai.azure.com/api/projects/ai-project-hackathon-team55
Model Deployment Name: gpt-4o
Current Python executable: c:\Users\Yu-Hong (Frank) Lin\Repos\azure-ai-agent-workshop\.venv\Scripts\python.exe


In [6]:
# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_AGENT_ENDPOINT"],  # Project endpoint from environment variable
    credential=DefaultAzureCredential(),      # Use Azure Default Credential for authentication
)

code_interpreter = CodeInterpreterTool()

with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="01_SDK agent",                        # Name of the agent
        instructions="You are a helpful agent",     # Instructions for the agent
        tools=code_interpreter.definitions,         # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",                                      # Role of the message sender
        content="What is the weather in Seattle today?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")
    
    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")
    
    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

Created agent, ID: asst_dANK6S5CN99uVK22PpC4TmwL
Created thread, ID: thread_4pn0PfzIQs78xgBkfSjVo5aD
Created message, ID: msg_1dvsQ2RsPyQEIEe6E0ltlCii
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "I can't fetch real-time data or access the internet to check the weather in Seattle today. You can check it on a reliable weather website like [Weather.com](https://www.weather.com/) or on a weather app. Let me know if you'd like advice on the best weather platforms or how to interpret the forecast!", 'annotations': []}}]
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'What is the weather in Seattle today?', 'annotations': []}}]


### 02 Listing AI Project Agents
This step demonstrates how to list all agents in your Azure AI Project using the Foundry SDK. The code retrieves and prints all available agents.

In [7]:
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_AGENT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# List the agents
with project_client:
    
    agent_list = project_client.agents.list_agents()
    for _agent in agent_list:
        print(_agent)

{'id': 'asst_dANK6S5CN99uVK22PpC4TmwL', 'object': 'assistant', 'created_at': 1754213572, 'name': '01_SDK agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}


### 03 Deleting an Agent from AI Project Client
This step shows how to delete an existing agent from your Azure AI Project using the Foundry SDK. The code demonstrates the deletion process.

In [8]:
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_AGENT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# Delete the agent when done
with project_client:
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Deleted agent


---

##### Reference: [Create a new Azure AI Foundry Agent Service project - Azure AI Foundry | Microsoft Learn](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/quickstart?pivots=programming-language-python-azure)